In [86]:
!pip install unidecode

import glob, os
import pandas as pd
import pathlib
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
import string
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Faradezo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Faradezo\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
def get_all_files(folder):
    file_list = []
    if os.path.exists(folder):
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_list.append(os.path.join(root,file))
    return file_list

In [15]:
def get_all_stop_words(df_col):
    #Faz a contagem de palavras por cada linha do artigo
    df_col['words'] = df_col['texto'].str.split().str.len()
    
    #Elimina qualquer artigo que tenha gerado menos de 100 palavras
    df_col = df_col.loc[(df_col['words'] > 100)]
    
    stop = stopwords.words('portuguese')
    # stop.append('nao')
    stop2 = list()
    for word in stop:
        stop2.append(unidecode(word))
    
    stemmer = nltk.stem.RSLPStemmer()
    # remoção de palavras de parada
    #df_col['texto_tratado'] = df_col['texto'].apply(lambda x: [item for item in x if item not in stop])
    df_col['texto_tratado'] = df_col['texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

    #Transforma em minuscula e remoção de acentos 
    df_col['texto_tratado'] = df_col['texto_tratado'].str.lower().apply(lambda x: unidecode(x))
    # remocação de pontuaçao
    df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.punctuation), ' ')
    # remocação de numeros
    df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.digits), '')
    # remoção de palavras de parada (repescagem)
    df_col['texto_tratado'] = df_col['texto_tratado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop2)]))
    
    
    vectorizer = CountVectorizer()
    XCOL = vectorizer.fit_transform(df_col['texto_tratado'])
    vocabulary_col = vectorizer.get_feature_names()
    pdXCOL = pd.DataFrame(data=XCOL.toarray(), columns=vocabulary_col) #.iloc[:,0::2]
    
    
    df_XCOL = pdXCOL.T
    df_XCOL['total'] = df_XCOL.sum(axis=1) 
    
    # Transpoe a matriz para que as palavras fiquem como linhas, e cada documento como uma linha
    df_XCOL = pdXCOL.T
    
    #Calcula a quantidade de vezes que cada palavra é utilizada
    df_XCOL['total'] = df_XCOL.sum(axis=1)
    
    #Remove as colunas individuais de cada documento para gerar a tabela
    df_XCOL.drop(df_XCOL.columns[0:len(df_col['texto'])], axis=1, inplace=True)
    
    #Ordena o resultado final
    df_XCOL = df_XCOL.sort_values(by='total', ascending=False)
    #return df_XCOL <- até aqui tava indo
    
    n_words = 30
    
    # Transpoe a matriz para que as palavras fiquem como linhas, e cada documento como uma linha
    pdXCOLT = pdXCOL.T
    
    #Mantem o percentual de frequencia de uso para comparação
    pdXCOLT['presente'] = (pdXCOLT.ne(0).sum(axis=1) -1) / len(df_col['texto'])
    
    #Ordena
    pdXCOLT = pdXCOLT.sort_values(by='presente', ascending=False)
    
    #Mantem so as 30 primeiras palavras
    pdXCOLT = pdXCOLT.head(n_words)
    
    # Apaga todas as clunas, exceto a com as palavras
    pdXCOLT.drop(pdXCOLT.columns[0:], axis=1, inplace=True)
    
    # Apaga todas as clunas, exceto a com as porcentagens presente
    #pdXCOLT.drop(pdXCOLT.columns[0:-1], axis=1, inplace=True)
    
    #gera o dicionario com o saco de palavras
    #ret = {}
    #for sac in pdXCOLT.to_dict().values():
    #    print(sac)
    #    ret = sac

    return pdXCOLT#ret

In [91]:
#lista dos colunistas disponiveis
for enu, colunista in enumerate(colunistas):
    print(colunista[7:])

alexandre_garcia.json
arnaldo_jabor.json
Carla_Araujo.json
Josias_de_Souza.json
Juca_Kfouri.json
Kennedy_Alencar.json
Leonardo_Sakamoto.json
Mauricio_Stycer.json
Reinaldo_Azevedo.json
Ricardo_Kotscho.json
Tales_Faria.json
Thays_Oyama.json
Walter_Maierovitch.json


In [87]:
sdp = {}
#variavel com o nome de todos os arquivos json dos colunistas q estao salvos
colunistas = get_all_files('textos')
for enu, colunista in enumerate(colunistas):
    link = (f'https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/{colunista[7:]}')
    df_col = pd.read_json(link)
    b = get_all_stop_words(df_col)
    if colunista[7:-5] not in sdp.keys():
        sdp[colunista[7:-5]] = b.T.columns.tolist()
sdp_file = open("sdp.pkl", "wb")
pickle.dump(sdp, sdp_file)
sdp_file.close()

C:\Users\Faradezo\AppData\Local\Temp/ipykernel_1516/3207719678.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_col['texto_tratado'] = df_col['texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
C:\Users\Faradezo\AppData\Local\Temp/ipykernel_1516/3207719678.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_col['texto_tratado'] = df_col['texto_tratado'].str.lower().apply(lambda x: unidecode(x))
C:\Users\Faradezo\AppData\Local\Temp/ipykernel_1516/3207719678.py:22

In [89]:
a_file = open("sdp.pkl", "rb")
output = pickle.load(a_file)
print(output)

{'alexandre_garcia': ['presidente', 'pais', 'anos', 'ainda', 'ser', 'agora', 'porque', 'sobre', 'vai', 'brasil', 'poder', 'contra', 'todos', 'ano', 'bolsonaro', 'governo', 'ter', 'maior', 'pode', 'assim', 'povo', 'semana', 'constituicao', 'quase', 'lei', 'brasileiros', 'milhoes', 'apenas', 'ministro', 'corrupcao'], 'arnaldo_jabor': ['tudo', 'ser', 'hoje', 'grande', 'mundo', 'vida', 'anos', 'pois', 'sempre', 'bem', 'nada', 'porque', 'pais', 'sobre', 'contra', 'ainda', 'assim', 'outro', 'onde', 'dia', 'agora', 'brasil', 'nunca', 'todos', 'tempo', 'politica', 'vez', 'diante', 'coisa', 'vai'], 'Carla_Araujo': ['presidente', 'bolsonaro', 'jair', 'ministro', 'feira', 'governo', 'nesta', 'economia', 'paulo', 'guedes', 'disse', 'coluna', 'federal', 'ministerio', 'afirmou', 'brasil', 'ainda', 'vai', 'tribunal', 'camara', 'congresso', 'segunda', 'ser', 'decisao', 'emergencial', 'sobre', 'apos', 'ter', 'economica', 'segundo'], 'Josias_de_Souza': ['bolsonaro', 'presidente', 'ser', 'sobre', 'minist